# Basic classification: Classify images of clothing

In [14]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print(tf.__version__)

2.11.0


## Import the Fashion_mnist data

In [16]:
fashion_mnist_train_1 = pd.read_csv("data/fashion-mnist-train-1.csv")
train_labels = fashion_mnist_train_1["label"]
train_images = fashion_mnist_train_1.drop(columns ="label")

fashion_mnist_train_2 = pd.read_csv("data/fashion-mnist-train-2.csv")
train_labels2 = fashion_mnist_train_2["label"]
train_images2 = fashion_mnist_train_2.drop(columns ="label")

fashion_mnist_test = pd.read_csv("data/fashion-mnist_test.csv")
test_labels = fashion_mnist_train_1["label"]
test_images = fashion_mnist_train_1.drop(columns ="label")

Loading the dataset returns four NumPy arrays:

* The `train_images` and `train_labels` arrays are the *training set*—the data the model uses to learn.
* The model is tested against the *test set*, the `test_images`, and `test_labels` arrays.

The images are 28x28 NumPy arrays, with pixel values ranging from 0 to 255. The *labels* are an array of integers, ranging from 0 to 9. These correspond to the *class* of clothing the image represents:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Each image is mapped to a single label. Since the *class names* are not included with the dataset, store them here to use later when plotting the images:

In [17]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Explore the data

Let's explore the format of the dataset before training the model. The following shows there are 60,000 images in the training set, with each image represented as 28 x 28 pixels:

In [18]:
fashion_mnist_train_1.shape

(14999, 785)

Likewise, there are 60,000 labels in the training set:

In [19]:
len(train_labels)

14999

In [20]:
len(train_images)

14999

Each label is an integer between 0 and 9:

In [21]:
train_labels

0        2
1        9
2        6
3        0
4        3
        ..
14994    4
14995    4
14996    0
14997    4
14998    2
Name: label, Length: 14999, dtype: int64

## Preprocess the data

The data must be preprocessed before training the network. If you inspect the first image in the training set, you will see that the pixel values fall in the range of 0 to 255:

Scale these values to a range of 0 to 1 before feeding them to the neural network model. To do so, divide the values by 255. It's important that the *training set* and the *testing set* be preprocessed in the same way:

In [22]:
train_images = train_images / 255.0
train_images2 = train_images2 / 255.0

To verify that the data is in the correct format and that you're ready to build and train the network, let's display the first 25 images from the *training set* and display the class name below each image.

## Build the model

Building the neural network requires configuring the layers of the model, then compiling the model.

### Set up the layers

The basic building block of a neural network is the [*layer*](https://www.tensorflow.org/api_docs/python/tf/keras/layers). Layers extract representations from the data fed into them. Hopefully, these representations are meaningful for the problem at hand.

Most of deep learning consists of chaining together simple layers. Most layers, such as `tf.keras.layers.Dense`, have parameters that are learned during training.

In [23]:
print(len(train_images))
print(len(train_images2))

14999
29999


In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

2023-01-28 23:47:08.707779: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


The first layer in this network, `tf.keras.layers.Flatten`, transforms the format of the images from a two-dimensional array (of 28 by 28 pixels) to a one-dimensional array (of 28 * 28 = 784 pixels). Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn; it only reformats the data.

After the pixels are flattened, the network consists of a sequence of two `tf.keras.layers.Dense` layers. These are densely connected, or fully connected, neural layers. The first `Dense` layer has 128 nodes (or neurons). The second (and last) layer returns a logits array with length of 10. Each node contains a score that indicates the current image belongs to one of the 10 classes.

### Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's [*compile*](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile) step:

* [*Loss function*](https://www.tensorflow.org/api_docs/python/tf/keras/losses) —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
* [*Optimizer*](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) —This is how the model is updated based on the data it sees and its loss function.
* [*Metrics*](https://www.tensorflow.org/api_docs/python/tf/keras/metrics) —Used to monitor the training and testing steps. The following example uses *accuracy*, the fraction of the images that are correctly classified.

In [25]:
model.compile(optimizer='adam',

              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Train the model

Training the neural network model requires the following steps:

1. Feed the training data to the model. In this example, the training data is in the `train_images` and `train_labels` arrays.
2. The model learns to associate images and labels.
3. You ask the model to make predictions about a test set—in this example, the `test_images` array.
4. Verify that the predictions match the labels from the `test_labels` array.


### Feed the model

To start training, call the [`model.fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) method—so called because it "fits" the model to the training data.

training the second train data_set because it contains more data.

In [ ]:
model.fit(train_images, train_labels, epochs=10)

Epoch 1/10
469/469 [==============================] - 1s 3ms/step - loss: 0.2570 - accuracy: 0.9111
Epoch 2/10
469/469 [==============================] - 1s 3ms/step - loss: 0.2567 - accuracy: 0.9101
Epoch 3/10
469/469 [==============================] - 1s 3ms/step - loss: 0.2562 - accuracy: 0.9123
Epoch 4/10
469/469 [==============================] - 2s 3ms/step - loss: 0.2555 - accuracy: 0.9105
Epoch 5/10
469/469 [==============================] - 1s 3ms/step - loss: 0.2558 - accuracy: 0.9114
Epoch 6/10
469/469 [==============================] - 1s 3ms/step - loss: 0.2546 - accuracy: 0.9123
Epoch 7/10
469/469 [==============================] - 1s 3ms/step - loss: 0.2538 - accuracy: 0.9115
Epoch 8/10
469/469 [==============================] - 1s 3ms/step - loss: 0.2532 - accuracy: 0.9128
Epoch 9/10
469/469 [==============================] - 1s 3ms/step - loss: 0.2528 - accuracy: 0.9126
Epoch 10/10
469/469 [==============================] - 1s 3ms/step - loss: 0.2524 - accuracy: 0.9122

As the model trains, the loss and accuracy metrics are displayed. This model reaches an accuracy of about 0.91 (or 91%) on the training data.

### Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

469/469 - 1s - loss: 18897.1621 - accuracy: 0.7441 - 612ms/epoch - 1ms/step

Test accuracy: 0.7441162467002869


It turns out that the accuracy on the test dataset is a little less than the accuracy on the training dataset. This gap between training accuracy and test accuracy represents *overfitting*. Overfitting happens when a machine learning model performs worse on new, previously unseen inputs than it does on the training data. An overfitted model "memorizes" the noise and details in the training dataset to a point where it negatively impacts the performance of the model on the new data. For more information, see the following:
*   [Demonstrate overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit#demonstrate_overfitting)
*   [Strategies to prevent overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit#strategies_to_prevent_overfitting)

### Make predictions

With the model trained, you can use it to make predictions about some images.
Attach a softmax layer to convert the model's linear outputs—[logits](https://developers.google.com/machine-learning/glossary#logits)—to probabilities, which should be easier to interpret.

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(test_images)

469/469 [==============================] - 1s 2ms/step


Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction:

A prediction is an array of 10 numbers. They represent the model's "confidence" that the image corresponds to each of the 10 different articles of clothing. You can see which label has the highest confidence value:

In [ ]:
model.save('model_train.h5')

In [ ]:
df1 = fashion_mnist_train_2.loc[:1]

df1.to_json("data/test.json")


## Training of the second model

In [26]:
model.fit(train_images2, train_labels2, epochs=10)

Epoch 1/10
938/938 [==============================] - 6s 4ms/step - loss: 0.5589 - accuracy: 0.8052
Epoch 2/10
938/938 [==============================] - 4s 4ms/step - loss: 0.4181 - accuracy: 0.8490
Epoch 3/10
938/938 [==============================] - 3s 3ms/step - loss: 0.3731 - accuracy: 0.8672
Epoch 4/10
938/938 [==============================] - 4s 5ms/step - loss: 0.3474 - accuracy: 0.8734
Epoch 5/10
938/938 [==============================] - 4s 4ms/step - loss: 0.3208 - accuracy: 0.8835
Epoch 6/10
938/938 [==============================] - 3s 4ms/step - loss: 0.3062 - accuracy: 0.8855
Epoch 7/10
938/938 [==============================] - 4s 4ms/step - loss: 0.2895 - accuracy: 0.8941
Epoch 8/10
938/938 [==============================] - 5s 5ms/step - loss: 0.2741 - accuracy: 0.8981
Epoch 9/10
938/938 [==============================] - 4s 5ms/step - loss: 0.2653 - accuracy: 0.9020
Epoch 10/10
938/938 [==============================] - 5s 5ms/step - loss: 0.2556 - accuracy: 0.9058

In [27]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

469/469 - 1s - loss: 32.2874 - accuracy: 0.8947 - 1s/epoch - 2ms/step

Test accuracy: 0.8947263360023499


In [28]:
model.save('model/model_train2.h5')